In [1]:
import numpy as np

In [8]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/dataset.csv')

# Extract features (X) and labels (y)
X = data[['x1', 'x2']].values
y = data['Label'].values

In [9]:
def train_test_split_manual(X, y, train_size=0.8):
    """
    Split the dataset into training and testing sets manually.

    Parameters:
    X (ndarray): The input feature data.
    y (ndarray): The target labels.
    train_size (float): The proportion of the dataset to include in the train split.

    Returns:
    X_train, X_test, y_train, y_test (tuple): The split data into training and testing sets.
    """
    # Shuffle the dataset
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    # Split the dataset
    train_index = int(train_size * X.shape[0])
    X_train, X_test = X[:train_index], X[train_index:]
    y_train, y_test = y[:train_index], y[train_index:]

    return X_train, X_test, y_train, y_test

In [10]:
# Manually split the data into training and testing sets
X_train_manual, X_test_manual, y_train_manual, y_test_manual = train_test_split_manual(X, y)

# Regide Regularization

In [11]:
def regularized_logistic_regression_cost_function(y, h, w, lambda_):
    """
    Regularized logistic regression cost function.

    Parameters:
    y (ndarray): The true labels.
    h (ndarray): The hypothesis function values (sigmoid activation outputs).
    w (ndarray): The weight vector of the model.
    lambda_ (float): The regularization parameter.

    Returns:
    cost (float): The regularized cost.
    """
    m = len(y)
    regularization_term = (lambda_ / (2 * m)) * np.sum(w[1:] ** 2)  # Exclude the bias term from regularization
    cost = (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) + regularization_term
    return cost


In [12]:
import numpy as np

def sigmoid_activation(x, w):

    z = np.dot(x, w)
    return 1 / (1 + np.exp(-z))

In [13]:
def regularized_gradient_of_cost_function(X, y, h, w, lambda_):
    """
    Gradient of the regularized logistic regression cost function.

    Parameters:
    X (ndarray): The input features.
    y (ndarray): The true labels.
    h (ndarray): The hypothesis function values (sigmoid activation outputs).
    w (ndarray): The weight vector of the model.
    lambda_ (float): The regularization parameter.

    Returns:
    gradient (ndarray): The gradient of the regularized cost function.
    """
    m = len(y)
    regularization_term = (lambda_ / m) * w
    regularization_term[0] = 0  # Exclude the bias term from regularization
    gradient = (1 / m) * np.dot(X.T, (h - y)) + regularization_term
    return gradient


In [14]:
def train_regularized_logistic_regression(X, y, learning_rate=0.01, iterations=1000, lambda_=0.1):
    """
    Train a regularized logistic regression model using gradient descent.

    Parameters:
    X (ndarray): The input feature data with an added bias column.
    y (ndarray): The target labels.
    learning_rate (float): The learning rate for gradient descent.
    iterations (int): The number of iterations to perform gradient descent.
    lambda_ (float): The regularization parameter.

    Returns:
    w (ndarray): The trained weights.
    """
    # Initialize weights with an additional term for the bias
    w = np.random.rand(X.shape[1] + 1)

    # Add a column of ones to X to account for the bias term (intercept)
    X = np.hstack((np.ones((X.shape[0], 1)), X))

    for i in range(iterations):
        # Calculate the hypothesis/sigmoid activations
        h = sigmoid_activation(X, w)

        # Calculate the gradient with regularization
        gradient = regularized_gradient_of_cost_function(X, y, h, w, lambda_)

        # Update the weights
        w -= learning_rate * gradient

    return w

    # Logistic regression prediction function
def predict_logistic_regression(X, w):
    """
    Make predictions using a logistic regression model.

    Parameters:
    X (ndarray): The input feature data.
    w (ndarray): The trained weights.

    Returns:
    y_pred (ndarray): The predicted labels.
    """
    # Add a column of ones to X to account for the bias term (intercept)
    X = np.hstack((np.ones((X.shape[0], 1)), X))

    # Calculate the sigmoid activations
    h = sigmoid_activation(X, w)

    # Convert probabilities to binary predictions
    y_pred = np.round(h)

    return y_pred


In [16]:
# Train the logistic regression model using our function
w_trained = train_regularized_logistic_regression(X, y, learning_rate=0.01, iterations=1000, lambda_=0.1)

# Make predictions on the training and testing sets
y_train_pred_manual = predict_logistic_regression(X_train_manual, w_trained)
y_test_pred_manual = predict_logistic_regression(X_test_manual, w_trained)

# Calculate the errors manually
train_error_manual = 1 - np.mean(y_train_pred_manual == y_train_manual)
test_error_manual = 1 - np.mean(y_test_pred_manual == y_test_manual)

train_error_manual, test_error_manual

(0.5106382978723405, 0.5416666666666667)

# Lasso regulariztion

In [17]:
def lasso_logistic_regression_cost_function(y, h, w, lambda_):
    """
    Lasso regularized logistic regression cost function.

    Parameters:
    y (ndarray): The true labels.
    h (ndarray): The hypothesis function values (sigmoid activation outputs).
    w (ndarray): The weight vector of the model.
    lambda_ (float): The regularization parameter.

    Returns:
    cost (float): The regularized cost.
    """
    m = len(y)
    regularization_term = (lambda_ / m) * np.sum(np.abs(w[1:]))  # Exclude the bias term from regularization
    cost = (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) + regularization_term
    return cost


In [18]:
def lasso_gradient_of_cost_function(X, y, h, w, lambda_):
    """
    Gradient of the Lasso regularized logistic regression cost function.

    Parameters:
    X (ndarray): The input features.
    y (ndarray): The true labels.
    h (ndarray): The hypothesis function values (sigmoid activation outputs).
    w (ndarray): The weight vector of the model.
    lambda_ (float): The regularization parameter.

    Returns:
    gradient (ndarray): The gradient of the regularized cost function.
    """
    m = len(y)
    regularization_term = np.sign(w) * (lambda_ / m)
    regularization_term[0] = 0  # Exclude the bias term from regularization
    gradient = (1 / m) * np.dot(X.T, (h - y)) + regularization_term
    return gradient


In [20]:
def train_lasso_logistic_regression(X, y, learning_rate=0.01, iterations=1000, lambda_=0.1):
    """
    Train a Lasso regularized logistic regression model using gradient descent.

    Parameters:
    X (ndarray): The input feature data with an added bias column.
    y (ndarray): The target labels.
    learning_rate (float): The learning rate for gradient descent.
    iterations (int): The number of iterations to perform gradient descent.
    lambda_ (float): The regularization parameter.

    Returns:
    w (ndarray): The trained weights.
    """
    # Initialize weights with an additional term for the bias
    w = np.random.rand(X.shape[1] + 1)

    # Add a column of ones to X to account for the bias term (intercept)
    X = np.hstack((np.ones((X.shape[0], 1)), X))

    for i in range(iterations):
        # Calculate the hypothesis/sigmoid activations
        h = sigmoid_activation(X, w)

        # Calculate the gradient with L1 regularization
        gradient = lasso_gradient_of_cost_function(X, y, h, w, lambda_)

        # Update the weights
        w -= learning_rate * gradient

    return w

def predict_logistic_regression(X, w):
    """
    Make predictions using a logistic regression model.

    Parameters:
    X (ndarray): The input feature data.
    w (ndarray): The trained weights.

    Returns:
    y_pred (ndarray): The predicted labels.
    """
    # Add a column of ones to X to account for the bias term (intercept)
    X = np.hstack((np.ones((X.shape[0], 1)), X))

    # Calculate the sigmoid activations
    h = sigmoid_activation(X, w)

    # Convert probabilities to binary predictions
    y_pred = np.round(h)

    return y_pred

In [21]:
# Train the logistic regression model using our function
w_trained = train_lasso_logistic_regression(X, y, learning_rate=0.01, iterations=1000, lambda_=0.1)

# Make predictions on the training and testing sets
y_train_pred_manual = predict_logistic_regression(X_train_manual, w_trained)
y_test_pred_manual = predict_logistic_regression(X_test_manual, w_trained)

# Calculate the errors manually
train_error_manual = 1 - np.mean(y_train_pred_manual == y_train_manual)
test_error_manual = 1 - np.mean(y_test_pred_manual == y_test_manual)

train_error_manual, test_error_manual

(0.4893617021276596, 0.5)

**We observed a notable enhancement in the accuracy of our logistic regression model upon the integration of L1 and L2 regularization. Prior to this adjustment, the model exhibited lower performance metrics on the same dataset. The regularization technique effectively mitigated the issue of overfitting, which is particularly beneficial in cases with numerous features or complex decision boundaries. By penalizing larger weights, the regularized model achieves a more generalized solution, leading to improved reliability and predictive accuracy. This improvement underscores the importance of regularization as a key technique in refining machine learning models, especially in scenarios prone to overfitting**